In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from nltk.sentiment import SentimentIntensityAnalyzer
import scipy.sparse as sp
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import nltk

In [2]:
# Télécharger les ressources nécessaires pour NLTK
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pnii\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pnii\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pnii\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
df = pd.read_csv('D:\\Projo\\Projet-Data-IA\\Merged_all_tables\\merged_final.csv', sep=',')
df.sort_values(by='averageRating', ascending=False, inplace=True)
df.drop(columns=['poster_path', 'backdrop_path', 'nconst_director'], inplace=True)

In [4]:
# Initialiser le stemmer et le lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Fonction pour appliquer le stemming et la lemmatisation à un texte
def preprocess_text(text):
    words = word_tokenize(text)
    words = [word for word in words if word.lower() not in stopwords.words('english')]
    stemmed_words = [stemmer.stem(word) for word in words]
    lemmatized_words = [lemmatizer.lemmatize(word) for word in stemmed_words]
    return ' '.join(lemmatized_words)

# Charger les données (Assurez-vous que df contient une colonne 'overview' avant d'exécuter ce code)
# df = pd.read_csv('votre_fichier.csv')

# Appliquer la fonction de prétraitement à la colonne "overview"
df['processed_overview'] = df['overview'].apply(preprocess_text)

# Définir les colonnes catégorielles et numériques
cat_cols = ["title", "genre1", "genre2", "production_companies_name", "Director_name", "Actors_Actresses", "processed_overview"]
num_cols = ["numVotes", "startYear", "runtimeMinutes", "popularity"]

# Créer une classe de transformation personnalisée pour extraire les sentiments de l'aperçu
class SentimentExtractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.analyzer = SentimentIntensityAnalyzer()

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        sentiments = []
        for overview in X["processed_overview"]:
            sentiment = self.analyzer.polarity_scores(overview)
            sentiments.append(list(sentiment.values()))
        return sentiments

# Créer un pipeline de transformation pour les caractéristiques
feature_pipeline = FeatureUnion([
    ("tfidf", TfidfVectorizer()),
    ("sentiment", SentimentExtractor())
])

# Appliquer le TfidfVectorizer sur la colonne "processed_overview"
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(df["processed_overview"])

# Appliquer le SentimentExtractor sur la colonne "processed_overview"
sentiment_extractor = SentimentExtractor()
X_sentiment = sentiment_extractor.fit_transform(df)

# Concaténer les résultats
X_features = sp.hstack((X_tfidf, X_sentiment), format='csr')

# Standardiser les colonnes numériques
scaler = StandardScaler()
X_numerical = scaler.fit_transform(df[num_cols])

# Spécifier les poids pour chaque colonne numérique
num_feature_weights = {
    "numVotes": 1,
    "startYear": 1,
    "runtimeMinutes": 1,
    "popularity": 2,
}

# Appliquer la pondération aux caractéristiques numériques en utilisant les poids spécifiés
X_numerical_weighted = X_numerical.copy()
for col, weight in num_feature_weights.items():
    col_index = num_cols.index(col)
    X_numerical_weighted[:, col_index] *= weight

# Créer un vecteur de poids pour les caractéristiques catégorielles
cat_feature_weights = {
    "title": 1,
    "genre1": 3,
    "genre2": 1,
    "production_companies_name": 2,
    "Director_name": 1,
    "Actors_Actresses": 2,
    "processed_overview": 2
}

# Créer un encodeur one-hot
encoder = OneHotEncoder(handle_unknown='ignore')

# Adapter l'encodeur aux données d'entraînement et le transformer
X_cat_encoded = encoder.fit_transform(df[cat_cols])

# Appliquer la pondération aux caractéristiques catégorielles en utilisant les poids spécifiés
for col, weight in cat_feature_weights.items():
    col_index = cat_cols.index(col)
    X_cat_encoded[:, col_index] *= weight

# Concaténer les caractéristiques encodées et standardisées
X = sp.hstack((X_features, X_numerical_weighted, X_cat_encoded), format='csr')

# Calculer la similarité cosinus entre les films
cosine_sim = cosine_similarity(X, X)

# Fonction de recommandation de films similaires
def recommend_movies(movie_title, cosine_sim=cosine_sim, df=df):
    # Vectoriser les titres de films
    title_vectorizer = TfidfVectorizer().fit_transform(df['title'])
    # Calculer la similarité cosinus entre le titre donné et les titres dans le dataset
    title_sim = cosine_similarity(title_vectorizer, title_vectorizer)
    # Trouver l'index du titre le plus similaire au titre donné
    title_idx = df[df['title'].str.lower() == movie_title.lower()].index
    if len(title_idx) == 0:
        # Si le titre exact n'est pas trouvé, utiliser la similarité cosinus pour trouver les titres les plus proches
        title_similarities = cosine_similarity(TfidfVectorizer().fit_transform([movie_title]), title_vectorizer)
        most_similar_title_idx = title_similarities.argsort()[0][-1]
        title_idx = [most_similar_title_idx]

    idx = title_idx[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    movie_indices = [i[0] for i in sim_scores]
    return df.iloc[movie_indices][['title']]  # Retourner les titres des films recommandés

# Exemple d'utilisation : obtenir des recommandations pour un film spécifique
movie_title = "Le parrain"
recommended_movies = recommend_movies(movie_title)
print("Recommandations pour", movie_title, ":")
print(recommended_movies)

Recommandations pour Le parrain :
                 title
1189      Forrest Gump
3729  Django Unchained
2616      Interstellar
4792          Parasite
3288         Inception
